In [1]:
import plotly.express as px
import numpy as np
import pandas as pd
from glob import glob
from pathlib import Path
from IPython.display import display, HTML
import plotly.graph_objects as go
import json
import os
import time

In [2]:
logs_path = Path("logs")
log_files = glob(str(logs_path / "*.logs.json"))
log_files.sort()
log_files

['logs/server_0.logs.json',
 'logs/worker_1.logs.json',
 'logs/worker_2.logs.json',
 'logs/worker_3.logs.json',
 'logs/worker_4.logs.json',
 'logs/worker_5.logs.json',
 'logs/worker_6.logs.json',
 'logs/worker_7.logs.json',
 'logs/worker_8.logs.json',
 'logs/worker_9.logs.json']

In [3]:
server_logs = log_files.pop(0)
server_logs, log_files

('logs/server_0.logs.json',
 ['logs/worker_1.logs.json',
  'logs/worker_2.logs.json',
  'logs/worker_3.logs.json',
  'logs/worker_4.logs.json',
  'logs/worker_5.logs.json',
  'logs/worker_6.logs.json',
  'logs/worker_7.logs.json',
  'logs/worker_8.logs.json',
  'logs/worker_9.logs.json'])

# Server

In [4]:
server_df = pd.read_json(server_logs, orient="index")
server_df["fid_score"] = server_df["fid_score"].apply(lambda x: 0 if x is None else x)
server_df["name"] = "server"
server_with_fid_df = server_df[server_df["fid_score"].notnull()]
display(server_df)
display(server_with_fid_df)

,end_time,elapsed_time,logging_time_end,received_time,sent_time,epoch,start_time,fid_calculation_time_start,fid_score,fid_calculation_time_end,name
0,2024-05-08 18:22:59.045585871,0.737381,1.715193e+09,2024-05-08 18:22:58.903584003,2024-05-08 18:22:58.415984869,0,2024-05-08 18:22:58.308205128,1.715193e+09,335.958466,1.715193e+09,server
1,2024-05-08 18:23:00.924341917,0.557929,1.715193e+09,2024-05-08 18:23:00.794301987,2024-05-08 18:23:00.415066957,1,2024-05-08 18:23:00.366413116,NaN,NaN,NaN,server
2,2024-05-08 18:23:01.591406107,0.666674,1.715193e+09,2024-05-08 18:23:01.422661066,2024-05-08 18:23:01.005789995,2,2024-05-08 18:23:00.924731970,NaN,NaN,NaN,server
3,2024-05-08 18:23:02.169080973,0.577257,1.715193e+09,2024-05-08 18:23:02.032815218,2024-05-08 18:23:01.643383026,3,2024-05-08 18:23:01.591823816,NaN,NaN,NaN,server
4,2024-05-08 18:23:02.900407076,0.730872,1.715193e+09,2024-05-08 18:23:02.747317076,2024-05-08 18:23:02.222375870,4,2024-05-08 18:23:02.169534922,NaN,NaN,NaN,server
...,...,...,...,...,...,...,...,...,...,...,...
56,2024-05-08 18:23:45.297830820,0.674372,1.715193e+09,2024-05-08 18:23:45.151446104,2024-05-08 18:23:44.667934895,56,2024-05-08 18:23:44.623459101,NaN,NaN,NaN,server
57,2024-05-08 18:23:45.947339058,0.647984,1.715193e+09,2024-05-08 18:23:45.820729017,2024-05-08 18:23:45.343509912,57,2024-05-08 18:23:45.299354792,NaN,NaN,NaN,server
58,2024-05-08 18:23:46.512902975,0.564069,1.715193e+09,2024-05-08 18:23:46.367815018,2024-05-08 18:23:45.999704123,58,2024-05-08 18:23:45.948833942,NaN,NaN,NaN,server
59,2024-05-08 18:23:47.083522081,0.569143,1.715193e+09,2024-05-08 18:23:46.958956003,2024-05-08 18:23:46.557200909,59,2024-05-08 18:23:46.514379025,NaN,NaN,NaN,server


,end_time,elapsed_time,logging_time_end,received_time,sent_time,epoch,start_time,fid_calculation_time_start,fid_score,fid_calculation_time_end,name
0,2024-05-08 18:22:59.045585871,0.737381,1.715193e+09,2024-05-08 18:22:58.903584003,2024-05-08 18:22:58.415984869,0,2024-05-08 18:22:58.308205128,1.715193e+09,335.958466,1.715193e+09,server
10,2024-05-08 18:23:07.336485147,0.782610,1.715193e+09,2024-05-08 18:23:07.109317064,2024-05-08 18:23:06.608719826,10,2024-05-08 18:23:06.553874969,1.715193e+09,327.022736,1.715193e+09,server
20,2024-05-08 18:23:15.257220030,0.732572,1.715193e+09,2024-05-08 18:23:15.098345995,2024-05-08 18:23:14.575732946,20,2024-05-08 18:23:14.524647950,1.715193e+09,311.138428,1.715193e+09,server
30,2024-05-08 18:23:23.016890049,0.598184,1.715193e+09,2024-05-08 18:23:22.873038054,2024-05-08 18:23:22.460731983,30,2024-05-08 18:23:22.418705940,1.715193e+09,329.682281,1.715193e+09,server
40,2024-05-08 18:23:31.565550089,0.672177,1.715193e+09,2024-05-08 18:23:31.367000103,2024-05-08 18:23:30.938243151,40,2024-05-08 18:23:30.893373013,1.715193e+09,307.568573,1.715193e+09,server
50,2024-05-08 18:23:39.846590042,1.089826,1.715193e+09,2024-05-08 18:23:39.645190954,2024-05-08 18:23:38.801666975,50,2024-05-08 18:23:38.756763935,1.715193e+09,309.392456,1.715193e+09,server
60,2024-05-08 18:23:47.737271070,0.651996,1.715193e+09,2024-05-08 18:23:47.587157965,2024-05-08 18:23:47.147497892,60,2024-05-08 18:23:47.085274935,1.715193e+09,362.259705,1.715193e+09,server


In [5]:
px.timeline(
    server_df,
    x_start="start_time",
    x_end="end_time",
    title="Server logs",
    template="plotly_white"
)

In [6]:
figure = go.Figure()
normalized_fid = server_with_fid_df.copy()
normalized_fid["fid_score"] = normalized_fid["fid_score"] / normalized_fid["fid_score"].max()
normalized_fid["elapsed_time"] = normalized_fid["elapsed_time"] / normalized_fid["elapsed_time"].max()
lineplot = px.line(
    normalized_fid,
    x="start_time",
    y="fid_score",
    title="Server logs",
)
duration_plot = px.bar(
    normalized_fid,
    x="start_time",
    y="elapsed_time",
    title="Server logs",
    text="elapsed_time",
    template="plotly_white"
)
figure.add_trace(lineplot.data[0])
figure.add_trace(duration_plot.data[0])
figure.show()

# Clients

In [7]:
def read_client_json(file_path: str):
    flattened_local_epochs = []
    flattened_swap = []
    flattened_send = []
    for outer_key, inner_dict in json.loads(open(file_path, "r").read()).items():
        root_attributes = {k: v for k, v in inner_dict.items() if not isinstance(v, dict)}
        if "start_time_swap" in root_attributes:
            flattened_swap.append({
                "outer_epoch": outer_key,
                "start_time": root_attributes["start_time_swap"],
                "end_time": root_attributes["end_time_swap"],
            })
        if "start_time_send" in root_attributes:
            flattened_send.append({
                "outer_epoch": outer_key,
                "start_time": root_attributes["start_time_send"],
                "end_time": root_attributes["end_time_send"],
            })
        for inner_key, values_dict in inner_dict.items():
            if not isinstance(values_dict, dict):
                continue
            flattened_local_epochs.append({
                "outer_epoch": outer_key,
                "local_epoch": inner_key,
                **values_dict,
                **root_attributes
            })

    # Convert flattened data to DataFrame
    name = os.path.basename(file_path).split(".")[0]
    
    df_local_epochs = pd.DataFrame(flattened_local_epochs)
    df_local_epochs["name"] = name

    df_swap = pd.DataFrame(flattened_swap)
    df_swap["name"] = f"{name}_swap"

    df_send = pd.DataFrame(flattened_send)
    df_send["name"] = f"{name}_send"

    return df_local_epochs, df_swap, df_send

In [8]:
clients_df = pd.concat([pd.concat(read_client_json(file)) for file in log_files])

clients_df["start_time"] = pd.to_datetime(clients_df["start_time"], unit="s")
clients_df["end_time"] = pd.to_datetime(clients_df["end_time"], unit="s")

server_to_client_df = server_df.copy()
server_to_client_df["name"] = "server_recv"
server_to_client_df["start_time"] = server_to_client_df["received_time"]
clients_df = pd.concat([clients_df, server_to_client_df])

server_to_client_df = server_df.copy()
server_to_client_df["name"] = "server_sent"
server_to_client_df["end_time"] = server_to_client_df["sent_time"]
clients_df = pd.concat([clients_df, server_to_client_df])

clients_df

,outer_epoch,local_epoch,d_loss_real,d_loss_fake,d_total_loss,elapsed_time,start_time,end_time,start_time_send,end_time_send,start_time_swap,end_time_swap,name,logging_time_end,received_time,sent_time,epoch,fid_calculation_time_start,fid_score,fid_calculation_time_end
0,0,0,7.551731e-01,0.618816,1.373989,0.398746,2024-05-08 18:22:58.347687006,2024-05-08 18:22:58.423476934,1.715193e+09,1.715193e+09,1.715193e+09,1.715193e+09,worker_1,NaN,NaT,NaT,NaN,NaN,NaN,NaN
1,0,9,1.164347e-04,0.460968,0.461085,0.398746,2024-05-08 18:22:58.717782974,2024-05-08 18:22:58.745877981,1.715193e+09,1.715193e+09,1.715193e+09,1.715193e+09,worker_1,NaN,NaT,NaT,NaN,NaN,NaN,NaN
2,1,0,1.790433e-04,0.469221,0.469400,0.360079,2024-05-08 18:23:00.384159088,2024-05-08 18:23:00.401314974,1.715193e+09,1.715193e+09,NaN,NaN,worker_1,NaN,NaT,NaT,NaN,NaN,NaN,NaN
3,1,9,1.192093e-08,0.032972,0.032972,0.360079,2024-05-08 18:23:00.707416058,2024-05-08 18:23:00.743683100,1.715193e+09,1.715193e+09,NaN,NaN,worker_1,NaN,NaT,NaT,NaN,NaN,NaN,NaN
4,2,0,0.000000e+00,0.054619,0.054619,0.410383,2024-05-08 18:23:00.939895153,2024-05-08 18:23:01.029250145,1.715193e+09,1.715193e+09,NaN,NaN,worker_1,NaN,NaT,NaT,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56,NaN,NaN,NaN,NaN,NaN,0.674372,2024-05-08 18:23:44.623459101,2024-05-08 18:23:44.667934895,NaN,NaN,NaN,NaN,server_sent,1.715193e+09,2024-05-08 18:23:45.151446104,2024-05-08 18:23:44.667934895,56.0,NaN,NaN,NaN
57,NaN,NaN,NaN,NaN,NaN,0.647984,2024-05-08 18:23:45.299354792,2024-05-08 18:23:45.343509912,NaN,NaN,NaN,NaN,server_sent,1.715193e+09,2024-05-08 18:23:45.820729017,2024-05-08 18:23:45.343509912,57.0,NaN,NaN,NaN
58,NaN,NaN,NaN,NaN,NaN,0.564069,2024-05-08 18:23:45.948833942,2024-05-08 18:23:45.999704123,NaN,NaN,NaN,NaN,server_sent,1.715193e+09,2024-05-08 18:23:46.367815018,2024-05-08 18:23:45.999704123,58.0,NaN,NaN,NaN
59,NaN,NaN,NaN,NaN,NaN,0.569143,2024-05-08 18:23:46.514379025,2024-05-08 18:23:46.557200909,NaN,NaN,NaN,NaN,server_sent,1.715193e+09,2024-05-08 18:23:46.958956003,2024-05-08 18:23:46.557200909,59.0,NaN,NaN,NaN


In [9]:
only_workers = [f"worker_{i+1}" for i in range(len(log_files))]
only_workers

['worker_1',
 'worker_2',
 'worker_3',
 'worker_4',
 'worker_5',
 'worker_6',
 'worker_7',
 'worker_8',
 'worker_9']

In [10]:
px.line(
    clients_df[clients_df["name"].isin(only_workers)],
    x="start_time",
    y="d_total_loss",
    color="name",
    title="Client logs",
    template="plotly_white"
)

In [11]:
clients_and_server_df = pd.concat(
    [
        clients_df[["start_time", "end_time", "name"]],
        server_df[["start_time", "end_time", "name"]],
    ]
)
clients_and_server_df

,start_time,end_time,name
0,2024-05-08 18:22:58.347687006,2024-05-08 18:22:58.423476934,worker_1
1,2024-05-08 18:22:58.717782974,2024-05-08 18:22:58.745877981,worker_1
2,2024-05-08 18:23:00.384159088,2024-05-08 18:23:00.401314974,worker_1
3,2024-05-08 18:23:00.707416058,2024-05-08 18:23:00.743683100,worker_1
4,2024-05-08 18:23:00.939895153,2024-05-08 18:23:01.029250145,worker_1
...,...,...,...
56,2024-05-08 18:23:44.623459101,2024-05-08 18:23:45.297830820,server
57,2024-05-08 18:23:45.299354792,2024-05-08 18:23:45.947339058,server
58,2024-05-08 18:23:45.948833942,2024-05-08 18:23:46.512902975,server
59,2024-05-08 18:23:46.514379025,2024-05-08 18:23:47.083522081,server


In [12]:
timeline = px.timeline(
    clients_and_server_df,
    x_start="start_time",
    x_end="end_time",
    color="name",
    title="Client logs",
    opacity=0.4,
    template="plotly_white"
)
# add vertical line to show when server logs started
for i in server_df["start_time"]:
    timeline.add_vline(x=i, line_dash="dash", line_color="red")
timeline.show()